# Imports


In [18]:

import uuid
import random

import duckdb
import llama_cpp

from langchain_text_splitters import RecursiveCharacterTextSplitter


# Ingest Data

## Load data from file

In [2]:
# read in data from file
file = "data/ep68.txt"
with open(file, 'r') as reader:
    texts = reader.read()

In [3]:
# check what data looks like
texts[:305]

" so chris last week of course the big news was claude sonnet 3.5 we said on that episode we would daily drive sonnet 3.5 for the entire week which i did and i know you did as well so what do you think now you've had one week of experience with claude sonnet well it's been a pleasure and i think i'm gonna"

Now let's split text into chunk. If you interested in how this specific splitter works see doc
[https://dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846](https://dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=300,
  chunk_overlap=100,
  length_function=len,
  is_separator_regex=False,
)

In [5]:
documents = text_splitter.create_documents([texts])
print(len(documents))

308


In [6]:
print(documents[0])
print("=" * 10)
print(documents[1])

page_content='so chris last week of course the big news was claude sonnet 3.5 we said on that episode we would daily drive sonnet 3.5 for the entire week which i did and i know you did as well so what do you think now you've had one week of experience with claude sonnet well it's been a pleasure and i think i'm'
page_content='now you've had one week of experience with claude sonnet well it's been a pleasure and i think i'm gonna steal steal your quote from the usage of it, which is it's so good at one shot solving problems, like just paste in what my thing is. And I always go fix plus or something really like simple'


## Generte embeddings for Vector search
Get vector representation of the texts we have

In [7]:
llm = llama_cpp.Llama(
  # download this model from here https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1/blob/7130e2d16051fdf3e0157e841f8b5a8d0d5e63ef/gguf/mxbai-embed-large-v1-f16.gguf 
  model_path="./models/mxbai-embed-large-v1-f16.gguf", 
  embedding=True, 
  verbose=False
)

In [8]:
%%time
embeddings = llm.create_embedding([item.page_content for item in documents])

CPU times: user 15min 39s, sys: 34.2 s, total: 16min 13s
Wall time: 2min 9s


In [9]:
documents_embeddings = [ 
  (document, embeddings['embedding'])  
  for document, embeddings in zip(documents, embeddings['data'])
]

## Upload data into 🦆

In [22]:
con = duckdb.connect("podcasts.duckdb")
con.sql("""CREATE TABLE podcast_transcript (
id STRING,
episode INTEGER,
paragraph INTEGER,
text STRING,
embeddings FLOAT[1024]
)""")

In [23]:
for index, (document, embeddings) in enumerate(documents_embeddings):
  con.execute(
    "INSERT INTO podcast_transcript VALUES (?, ?, ?, ?, ?)", 
    [str(uuid.uuid4()), 68, index, document.page_content, embeddings]
  )

Install 🦆 [Full-text search](https://duckdb.org/docs/extensions/full_text_search.html) extention

In [24]:
con.sql("INSTALL fts")
con.sql("LOAD fts")


The extension adds two `PRAGMA` statements to DuckDB: one to create, and one to drop an index. Additionally, a scalar macro stem is added, which is used internally by the extension.

The PRAGMA statement is a special SQL extension adopted by DuckDB from SQLite and is used to interact with external libs in DuckDB

So let's create an index for full-text search [(doc)](https://duckdb.org/docs/extensions/full_text_search.html#pragma-create_fts_index)

In [25]:
con.sql("""
PRAGMA create_fts_index(
    'podcast_transcript', 'id', 'text', overwrite=1
);
""")

Now let's look into our data

In [26]:
con.sql("SELECT * FROM podcast_transcript").show(max_rows=10)

┌──────────────────────┬─────────┬───────────┬──────────────────────┬──────────────────────────────────────────────────┐
│          id          │ episode │ paragraph │         text         │                    embeddings                    │
│       varchar        │  int32  │   int32   │       varchar        │                   float[1024]                    │
├──────────────────────┼─────────┼───────────┼──────────────────────┼──────────────────────────────────────────────────┤
│ 813eb6d7-5614-4206…  │      68 │         0 │ so chris last week…  │ [0.64156187, 0.67421854, 0.1992178, -0.1325478…  │
│ 002fc5eb-48d7-4875…  │      68 │         1 │ now you've had one…  │ [-0.33717906, 0.12142487, 0.18780196, -0.12293…  │
│ e5fd8656-0d9a-46f6…  │      68 │         2 │ like just paste in…  │ [0.11697711, 0.37458858, -0.09488027, 0.310543…  │
│ 2c4563e5-677e-41bf…  │      68 │         3 │ the funny thing is…  │ [0.6980952, 0.22205475, 0.18904458, 0.73308444…  │
│ 55767b5e-2321-4fae…  │      68

In [27]:
con.sql("SELECT COUNT(id), array_length(embeddings) FROM podcast_transcript GROUP BY array_length(embeddings)").show(max_rows=10)

┌───────────┬──────────────────────────┐
│ count(id) │ array_length(embeddings) │
│   int64   │          int64           │
├───────────┼──────────────────────────┤
│       308 │                     1024 │
└───────────┴──────────────────────────┘



# Full-text search
Full-Text Search works by first breaking down a document or text into words and then indexing each word. 
The index is essentially a catalog of all the words in the document, along with the location of each word within the document. 
When a user enters a search query, the system compares it against the indexed words and returns a list of relevant documents 
or results.

Full-Text Search also includes various algorithms and techniques that improve search results. 
For example, stemming algorithms are used to match different forms of a word, 
such as "run," "running," and "ran". Stop words, which are common words like "the," "and," and "of," 
are also removed from the index to improve search efficiency. In our case [create_fts_index](https://duckdb.org/docs/extensions/full_text_search.html#pragma-create_fts_index) function uses default pre-defined list of 571 English stopwords, but you can change it using `stopwords` param. It also deals with different forms of words (see [Stemmer and Stopwords sections in the blog post](https://motherduck.com/blog/search-using-duckdb-part-3/) for more details)

In our case we will use [match_bm25](https://duckdb.org/docs/extensions/full_text_search.html#match_bm25-function) function for the scoring. BM25 (BM is an abbreviation of best matching) is a ranking function used by search engines to estimate the relevance of documents to a given search query. It checks how often specific words can be found in the document and taking into account length of the document and average length of the documents. You can find formula here, it looks not so difficult [FORMULA](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function)



In [34]:
def fts(query, limit=3):
  return con.sql("""
    SELECT text, paragraph, fts_main_podcast_transcript.match_bm25(
            id,
            $searchTerm
        ) AS score
    FROM podcast_transcript
    WHERE score IS NOT NULL
    ORDER BY score DESC
    LIMIT $limit
    """, params={"searchTerm": query, "limit": limit})

# Vector search

In the world of Natural Language Processing (NLP), Vector Embeddings, or vector search, refer to the numerical representations of textual data. These embeddings transform words, phrases, or even entire documents into vectors of real numbers, that capture word relationships and semantic meaning of the textual data. Representing text as vector embeddings enables the possibility of applying mathematical operations such as similarity comparison, clustering, and classification.

We wil use simple [array_cosine_similarity](https://duckdb.org/docs/sql/functions/array.html#array_cosine_similarityarray1-array2)
Cosine Similarity is a metric for calculating the semantic similarity of vector embeddings. It is also commonly used in the semantic retrieval of information. We calculate it by taking the dot product of the two normalized vectors.

1. A value of 1 for this metric indicates that the two vectors are identical
2. A value of 0 means they are independent (orthogonal)
3. A value of -1 indicates that they are diametrically opposed (opposites)


In [31]:
similarity_checklist = [
    "king",
    "queen",
    "король",
    "тура",
    "слон",
    "woman",
    "man",
    "cat",
    "meow",
    "lion",
    "dog",
    "wolf"
]
con.sql("DROP TABLE similarity_check")
con.sql("""CREATE TABLE IF NOT EXISTS similarity_check (
id STRING,
word STRING,
embedding FLOAT[1024]
)""")

similarity_check_embeddings = llm.create_embedding(similarity_checklist)


for word, embedding in zip(similarity_checklist, similarity_check_embeddings['data']):
  con.execute(
    "INSERT INTO similarity_check VALUES (?, ?, ?)", 
    [str(uuid.uuid4()), word, embedding['embedding']]
  )

In [29]:
con.sql("SELECT * FROM similarity_check").show(max_rows=10)


┌──────────────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────┐
│          id          │  word   │                                      embedding                                      │
│       varchar        │ varchar │                                     float[1024]                                     │
├──────────────────────┼─────────┼─────────────────────────────────────────────────────────────────────────────────────┤
│ 1af59b71-3437-4734…  │ king    │ [0.38659263, -0.5397655, 0.27348864, -0.20799291, -0.33552152, 0.024020072, -0.45…  │
│ 83b2b191-87cf-46da…  │ queen   │ [-0.028366804, -0.5756435, 0.10505161, -0.107996225, -0.2769209, -0.19957519, 0.4…  │
│ 0de097bf-2852-48d0…  │ король  │ [0.60744846, -0.43981713, -0.4478092, -0.04732138, 0.14943007, -0.6030525, -0.353…  │
│ 703034bf-d93c-4dbf…  │ тура    │ [-0.23034039, -0.15377502, -0.5754214, -0.39095268, -0.19352104, -0.3595555, 0.43…  │
│ 8bf0c618-5793-493a…  │ woman  

In [32]:
con.sql("""SELECT x.word as word_1,
       y.word as word_2,
       array_cosine_similarity(x.embedding, y.embedding) AS similarity_metric
    FROM similarity_check AS x
    CROSS JOIN similarity_check AS y
    WHERE word_1 > word_2
    ORDER BY similarity_metric DESC;
""").show(max_rows=100)


┌─────────┬─────────┬───────────────────┐
│ word_1  │ word_2  │ similarity_metric │
│ varchar │ varchar │       float       │
├─────────┼─────────┼───────────────────┤
│ meow    │ cat     │        0.82352495 │
│ wolf    │ dog     │        0.73758084 │
│ lion    │ cat     │         0.7090323 │
│ queen   │ king    │         0.6937217 │
│ dog     │ cat     │          0.691271 │
│ wolf    │ lion    │        0.67236227 │
│ wolf    │ cat     │         0.6530658 │
│ lion    │ dog     │        0.65172124 │
│ man     │ king    │        0.62940687 │
│ woman   │ queen   │         0.6185585 │
│ woman   │ man     │        0.61193275 │
│ king    │ cat     │         0.6067609 │
│ queen   │ lion    │         0.6052481 │
│ слон    │ король  │        0.60499555 │
│ man     │ dog     │         0.6042128 │
│ lion    │ king    │        0.59995544 │
│ wolf    │ king    │        0.59787065 │
│ queen   │ cat     │         0.5970367 │
│ man     │ cat     │        0.59703016 │
│ meow    │ lion    │         0.59

In [36]:
def vector_search(query, limit=3):
  search_vector = llm.create_embedding([query])['data'][0]['embedding']
  return con.sql("""
    SELECT text, paragraph, 
           array_cosine_similarity(
             embeddings, $searchVector::FLOAT[1024]
           ) AS score
    FROM podcast_transcript
    ORDER BY score DESC
    LIMIT $limit
    """, params={"searchVector": search_vector, "limit": limit})

# Testing

In [37]:
query = "Who did Olympics recap?"
print("FTS")
display(fts(query))
print("=" * 50)
print("VECTORE")
display(vector_search(query))

FTS


┌─────────────────────────────────────────────────────────────────────────────────────┬───────────┬────────────────────┐
│                                        text                                         │ paragraph │       score        │
│                                       varchar                                       │   int32   │       double       │
├─────────────────────────────────────────────────────────────────────────────────────┼───────────┼────────────────────┤
│ like replicating things, listen to this. Hi, Kelly. Welcome to your daily Olympic…  │       205 │ 3.7771193989684075 │
│ you've made enough now. Just play the good ones. I was going to get to this later…  │       204 │ 3.6331466774026557 │
│ head right to the pool. Team USA secured a stunning victory in the men's four by …  │       206 │ 2.1472140417770316 │
└─────────────────────────────────────────────────────────────────────────────────────┴───────────┴────────────────────┘

VECTORE


┌─────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬────────────┐
│                                            text                                             │ paragraph │   score    │
│                                           varchar                                           │   int32   │   float    │
├─────────────────────────────────────────────────────────────────────────────────────────────┼───────────┼────────────┤
│ like replicating things, listen to this. Hi, Kelly. Welcome to your daily Olympic recap, …  │       205 │  0.6540807 │
│ head right to the pool. Team USA secured a stunning victory in the men's four by one. So …  │       206 │ 0.61149085 │
│ you've made enough now. Just play the good ones. I was going to get to this later with th…  │       204 │   0.572537 │
└─────────────────────────────────────────────────────────────────────────────────────────────┴───────────┴────────────┘

In [38]:
fts(query).show(max_width=10000)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬────────────────────┐
│                                                                                                                                                    text                                                                                                                                                    │ paragraph │       score        │
│                                                                                                                                                  varchar                                                                                                                                                   │   int32   │       double 

In [40]:
vector_search(query).show(max_width=10000)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬────────────┐
│                                                                                                                                                    text                                                                                                                                                    │ paragraph │   score    │
│                                                                                                                                                  varchar                                                                                                                                                   │   int32   │   float    │
├───────────────

# Hyrbid search
To be continue...

# Some links
1. Full text and vectore search in DuckDB video https://www.youtube.com/watch?v=U6k7HnQiWzc
2. Hybrid search in DuckDB video https://www.youtube.com/watch?v=Phg91R_RLLE
4. Hybrid Search in DuckDB Blog post https://motherduck.com/blog/search-using-duckdb-part-3/
5. Vector Search in DuckDB Blog post https://motherduck.com/blog/search-using-duckdb-part-1/
